In [13]:
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
import os
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
# response = requests.get('https://www.merit-times.com.tw/NewsPage.aspx?unid=903451')
response = requests.get('https://rate.bot.com.tw/xrt?Lang=zh-TW')
soup = BeautifulSoup(response.text,"html.parser")
body_content = soup.body
body_content_str = str(body_content)
soup1 = BeautifulSoup(body_content_str,'html.parser')
for script_or_style in soup(['script','style']):
    script_or_style.extract()

cleaned_content = soup.get_text(separator="\n")
cleaned_content = "\n".join(
    line.strip() for line in cleaned_content.splitlines() if line.strip()
)
#print(cleaned_content)
#print(len([cleaned_content[i : i + 6000] for i in range(0, len(cleaned_content), 6000)]))

model = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''
    你的任務是取出指定的內容,並輸出文字
    請依照下面的指示:
    樣本:
    ```
    澳幣 (AUD)
    澳幣 (AUD)
    20.15
    20.93
    20.365
    20.71
    ```
    上面的樣本代表:
    澳幣(AUD):
    現金匯率(本行買入):20.15
    現金匯率(本行賣出):20.93
    即期匯率(本行買入):20.365
    即期匯率(本行賣出):20.365    
    '''
)

response = model.generate_content(cleaned_content)
result_text = response.text
print(result_text)




美金(USD):
現金匯率(本行買入):32.28
現金匯率(本行賣出):32.95
即期匯率(本行買入):32.605
即期匯率(本行賣出):32.755
港幣(HKD):
現金匯率(本行買入):4.04
現金匯率(本行賣出):4.244
即期匯率(本行買入):4.161
即期匯率(本行賣出):4.231
英鎊(GBP):
現金匯率(本行買入):39.66
現金匯率(本行賣出):41.78
即期匯率(本行買入):40.555
即期匯率(本行賣出):41.185
澳幣(AUD):
現金匯率(本行買入):20.2
現金匯率(本行賣出):20.98
即期匯率(本行買入):20.415
即期匯率(本行賣出):20.76
加拿大幣(CAD):
現金匯率(本行買入):22.28
現金匯率(本行賣出):23.19
即期匯率(本行買入):22.61
即期匯率(本行賣出):22.94
新加坡幣(SGD):
現金匯率(本行買入):23.75
現金匯率(本行賣出):24.66
即期匯率(本行買入):24.22
即期匯率(本行賣出):24.44
瑞士法郎(CHF):
現金匯率(本行買入):35.59
現金匯率(本行賣出):36.79
即期匯率(本行買入):36.2
即期匯率(本行賣出):36.59
日圓(JPY):
現金匯率(本行買入):0.2031
現金匯率(本行賣出):0.2159
即期匯率(本行買入):0.2099
即期匯率(本行賣出):0.2149
南非幣(ZAR):
現金匯率(本行買入):-
現金匯率(本行賣出):-
即期匯率(本行買入):1.707
即期匯率(本行賣出):1.797
瑞典幣(SEK):
現金匯率(本行買入):-
現金匯率(本行賣出):-
即期匯率(本行買入):2.93
即期匯率(本行賣出):3.05
紐元(NZD):
現金匯率(本行買入):18.14
現金匯率(本行賣出):18.99
即期匯率(本行買入):18.47
即期匯率(本行賣出):18.77
泰幣(THB):
現金匯率(本行買入):0.8432
現金匯率(本行賣出):1.0332
即期匯率(本行買入):0.9536
即期匯率(本行賣出):0.9996
菲國比索(PHP):
現金匯率(本行買入):0.4942
現金匯率(本行賣出):0.6262
即期匯率(本行買入):-
即期匯率(本行賣出):-
印尼幣

In [26]:
import google.generativeai as genai
import typing_extensions as typing

class Rate(typing.TypedDict):
    buy:float | None
    sell:float | None

class Recipe(typing.TypedDict):
    幣別:str
    類別1:Rate
    類別2:Rate

genai.configure(api_key=os.environ['GEMINI_API_KEY'])

model1 = genai.GenerativeModel(
    "gemini-2.0-flash-exp",
    system_instruction='''    
    1.如果無法轉成float,請使用None替代
    **輸入的格式:**
    ```
    澳幣(AUD):
    現金匯率(本行買入):20.15
    現金匯率(本行賣出):20.93
    即期匯率(本行買入):20.365
    即期匯率(本行賣出):20.71 
    ```

    **轉換輸入的格式成為輸出json schema:**
    ```
    [
        {
           '幣別':'澳幣',
           '類別1':{
                    'buy':20.15,
                    'sell':20.93
                }
           '類別2':{
                    'buy':20.365,
                    'sell':20.71
                } 
        }
    ]
    ```
    '''
)
response =model.generate_content(
    result_text,
    generation_config = genai.GenerationConfig(
        response_mime_type="application/json",
        response_schema=list[Recipe]
    ) 
)
print(response.text)


[
  {
    "幣別": "美金(USD)",
    "類別1": {
      "buy": 32.28,
      "sell": 32.95
    },
    "類別2": {
      "buy": 32.605,
      "sell": 32.755
    }
  },
  {
    "幣別": "港幣(HKD)",
    "類別1": {
      "buy": 4.04,
      "sell": 4.244
    },
    "類別2": {
      "buy": 4.161,
      "sell": 4.231
    }
  },
  {
    "幣別": "英鎊(GBP)",
    "類別1": {
      "buy": 39.66,
      "sell": 41.78
    },
    "類別2": {
      "buy": 40.555,
      "sell": 41.185
    }
  },
  {
    "幣別": "澳幣(AUD)",
    "類別1": {
      "buy": 20.2,
      "sell": 20.98
    },
    "類別2": {
      "buy": 20.415,
      "sell": 20.76
    }
  },
  {
    "幣別": "加拿大幣(CAD)",
     "類別1": {
      "buy": 22.28,
      "sell": 23.19
    },
    "類別2": {
      "buy": 22.61,
      "sell": 22.94
    }
  },
  {
    "幣別": "新加坡幣(SGD)",
    "類別1": {
      "buy": 23.75,
      "sell": 24.66
    },
    "類別2": {
      "buy": 24.22,
      "sell": 24.44
    }
  },
    {
    "幣別": "瑞士法郎(CHF)",
     "類別1": {
      "buy": 35.59,
      "sell": 36.79
    },
    "類